In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# # 
# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

import matplotlib.pyplot as plt
from PIL import Image
import random, os
import numpy as np

In [ ]:

import tensorflow as tf
import keras
from keras.layers import Input, Dense, Flatten
# from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


In [ ]:
!pip install keras keras.preprocessing.image

In [ ]:

#train and test data directory
data_dir = "/kaggle/input/hmmmmm/data/train"
test_data_dir = "/kaggle/input/hmmmmm/data/test"

In [ ]:
import os

def get_image_paths(data_dir, valid_extensions=("jpg", "jpeg", "png", "bmp", "tif")):
    """
    Extracts image file paths from the given directory and its subdirectories.
    
    Parameters:
        data_dir (str): Path to the dataset directory.
        valid_extensions (tuple): Valid image file extensions.
    
    Returns:
        List[str]: A list of image file paths.
    """
    image_paths = []
    
    # Walk through all subdirectories
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.lower().endswith(valid_extensions):
                image_paths.append(os.path.join(root, file))
    
    return image_paths

# Define dataset directories
data_dir = "/kaggle/input/hmmmmm/data/train"
test_data_dir = "/kaggle/input/hmmmmm/data/test"

# Get image paths
train_images = get_image_paths(data_dir)
test_images = get_image_paths(test_data_dir)

# Print some results
print(f"Total training images: {len(train_images)}")
print(f"Total test images: {len(test_images)}")
print("Example train image path:", train_images[0] if train_images else "No images found")
print("Example test image path:", test_images[0] if test_images else "No images found")


In [ ]:
class ImageFolderEX(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except:
            return None
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

In [ ]:


transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor()])
 
test_ds = ImageFolderEX(test_data_dir, transform = transform)
 
print('test samples:', len(test_ds))

In [ ]:
#load the train and validation into batches
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

test_dl = DataLoader(test_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [ ]:
# transform = transforms.Compose([transforms.Resize((224,224)),
#                                 transforms.ToTensor()])

# # #load the train and test data
# # dataset = ImageFolderEX(data_dir, transform = transform)
# test_ds = ImageFolderEX(test_data_dir, transform = transform)

# print('dataset samples:', len(dataset))
# print('test samples:', len(test_ds))
# print("Classes are:", dataset.classes)

# img, label = dataset[0]
# print(f'Example: first sample size: {img.shape}, and label: {label}')

In [ ]:
# print(dataset[0])

In [ ]:
tf.keras.utils.set_random_seed(12)

batch_size = 32
img_height = img_width =224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="binary",
    #class_names=None,
    shuffle=True,
    seed=12,
    validation_split= 0.15,
    subset="training",
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:


import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
class_names = train_ds.class_names
for images, labels in train_ds.take(2):
    for i in range(32):
        ax = plt.subplot(6, 6, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[int(labels[i])])
        plt.axis("off")



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
# Function to apply edge detection
def apply_edge_detection(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  
    if img is None:
        print(f"Error: Unable to load image {image_path}")
        return  # Stop execution if the image is not loaded
    else:
        print(f"Successfully loaded image: {image_path}")
    # Read image in grayscale

    # Apply different edge detection techniques
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobelx, sobely)

    prewittx = cv2.filter2D(img, -1, np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]]))
    prewitty = cv2.filter2D(img, -1, np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]]))
    prewitt = cv2.magnitude(prewittx.astype(np.float32), prewitty.astype(np.float32))

    robertsx = cv2.filter2D(img, -1, np.array([[1, 0], [0, -1]]))
    robertsy = cv2.filter2D(img, -1, np.array([[0, 1], [-1, 0]]))
    roberts = cv2.magnitude(robertsx.astype(np.float32), robertsy.astype(np.float32))

    canny = cv2.Canny(img, 100, 200)

    # Display results
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 3, 1), plt.imshow(img, cmap='gray'), plt.title("Original")
    plt.subplot(2, 3, 2), plt.imshow(sobel, cmap='gray'), plt.title("Sobel Edge Detection")
    plt.subplot(2, 3, 3), plt.imshow(prewitt, cmap='gray'), plt.title("Prewitt Edge Detection")
    plt.subplot(2, 3, 4), plt.imshow(roberts, cmap='gray'), plt.title("Roberts Edge Detection")
    plt.subplot(2, 3, 5), plt.imshow(canny, cmap='gray'), plt.title("Canny Edge Detection")
    plt.show()

# # Apply to a sample image
# sample_image = os.path.join(train_images, os.listdir(train_images)[0])
apply_edge_detection((train_images)[0])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# from tensorflow.keras.applications import EfficientNetB3
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(256,256,3))
# base_model.trainable = False  # Freeze pretrained weights

# model = Sequential([
#     base_model,
#     GlobalAveragePooling2D(),
#     Dense(128, activation='relu'),
#     Dense(1, activation='sigmoid')  # Binary classification
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# from skimage.feature import hog
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split

# def extract_hog_features(image_path):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     image = cv2.resize(image, (256,256))
#     features, _ = hog(image, pixels_per_cell=(16,16), cells_per_block=(2,2), visualize=True)
#     return features

# # Extract HOG features for all images
# X = [extract_hog_features(img) for img in train_images]
# y = [0 if "notinfected" in img else 1 for img in train_images]

# # Train an SVM model
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = SVC(kernel='rbf')
# model.fit(X_train, y_train)
# print("SVM Accuracy:", model.score(X_test, y_test))


In [ ]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model
# from xgboost import XGBClassifier

# # Load pre-trained model for feature extraction
# base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(256,256,3))
# feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# # Extract deep learning features
# X = [feature_extractor.predict(cv2.resize(cv2.imread(img), (256,256)).reshape(1,256,256,3)).flatten() for img in train_images]
# y = [0 if "notinfected" in img else 1 for img in train_images]

# # Train an XGBoost classifier
# model = XGBClassifier()
# model.fit(X, y)
# print("XGBoost Accuracy:", model.score(X, y))


In [ ]:
enhanced_directory = "/kaggle/working/enhanced_data"  # folder for enhanced dataset


# Create directory for saving enhanced images
if not os.path.exists(enhanced_directory):
    os.makedirs(enhanced_directory)



In [ ]:
# import os
# import cv2
# import numpy as np
# import random
# import shutil
# from tqdm import tqdm
# from skimage import exposure
# from skimage.filters import unsharp_mask

# # Original dataset path
# data_dir = "/kaggle/input/hmmmmm/data/train"
# enhanced_directory = "/kaggle/working/enhanced_data"  # Save enhanced images

# # Create directory for enhanced images
# if not os.path.exists(enhanced_directory):
#     os.makedirs(enhanced_directory)

# # Define augmentation functions
# def image_translation(img, tx, ty):
#     M = np.float32([[1, 0, tx], [0, 1, ty]])
#     return cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

# def image_reflection(img):
#     return cv2.flip(img, 1)  # Flip horizontally

# def image_rotation(img, angle):
#     h, w = img.shape[:2]
#     M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
#     return cv2.warpAffine(img, M, (w, h))

# def image_scaling(img, scale):
#     h, w = img.shape[:2]
#     return cv2.resize(img, (int(w * scale), int(h * scale)))

# def image_cropping(img, x1, y1, x2, y2):
#     return img[y1:y2, x1:x2]

# def image_shear_x(img, shear_factor):
#     h, w = img.shape[:2]
#     M = np.array([[1, shear_factor, 0], [0, 1, 0]], dtype=np.float32)
#     return cv2.warpAffine(img, M, (w, h))

# def image_shear_y(img, shear_factor):
#     h, w = img.shape[:2]
#     M = np.array([[1, 0, 0], [shear_factor, 1, 0]], dtype=np.float32)
#     return cv2.warpAffine(img, M, (w, h))

# # Multi-Scale Retinex Enhancement
# def multi_scale_retinex(img, sigma_list=[15, 80, 250], gain=128, offset=128):
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img = np.float32(img) + 1.0

#     retinex = np.zeros_like(img)
#     for sigma in sigma_list:
#         retinex += np.log1p(img) - np.log1p(cv2.GaussianBlur(img, (0, 0), sigma))

#     retinex = (retinex - np.min(retinex)) / (np.max(retinex) - np.min(retinex)) * gain + offset
#     retinex = np.clip(retinex, 0, 255).astype(np.uint8)
    
#     return cv2.cvtColor(retinex, cv2.COLOR_GRAY2BGR)

# # Process images
# for category in ["infected", "notinfected"]:
#     category_path = os.path.join(data_dir, category)
#     enhanced_category_path = os.path.join(enhanced_directory, category)

#     # Create directories
#     if not os.path.exists(enhanced_category_path):
#         os.makedirs(enhanced_category_path)

#     for img_name in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
#         img_path = os.path.join(category_path, img_name)
#         img = cv2.imread(img_path)

#         if img is None:
#             continue  # Skip unreadable images

#         # Apply augmentations
#         transformed_images = [
#             image_translation(img, random.randint(-50, 50), random.randint(-30, 30)),
#             image_reflection(img),
#             image_rotation(img, random.randint(-45, 45)),
#             image_scaling(img, random.uniform(0.8, 1.2)),
#             image_cropping(img, 50, 50, img.shape[1]-50, img.shape[0]-50),
#             image_shear_x(img, random.uniform(-0.3, 0.3)),
#             image_shear_y(img, random.uniform(-0.3, 0.3)),
#         ]

#         # Save augmented images
#         for i, aug_img in enumerate(transformed_images):
#             aug_img_path = os.path.join(enhanced_category_path, f"{img_name}_aug_{i}.jpg")
#             cv2.imwrite(aug_img_path, aug_img)

#         # Apply MSR enhancement and save
#         enhanced_img = multi_scale_retinex(img)
#         enhanced_img_path = os.path.join(enhanced_category_path, f"{img_name}_enhanced.jpg")
#         cv2.imwrite(enhanced_img_path, enhanced_img)

# print("✅ Dataset augmentation and enhancement complete! 🚀")


In [ ]:
# len(enhanced_dataset)

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Define dataset path
data_dir = "/kaggle/input/hmmmmm/data/train"

# Choose a category (infected or notinfected)
category = "infected"  # Change to "notinfected" if needed

# Get a sample image path
sample_image_path = os.path.join(data_dir, category, os.listdir(os.path.join(data_dir, category))[0])

# Read image using OpenCV
img = cv2.imread(sample_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for Matplotlib

# Display the image
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis("off")
plt.title(f"Sample Image from '{category}' class")
plt.show()


In [ ]:
!pip install torch torchvision timm transformers datasets


In [ ]:
# import os
# import torch
# import torchvision.transforms as transforms
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader
# from torchvision import models
# import timm

# # Dataset Paths
# train_data_dir = "/kaggle/input/hmmmmm/data/train"
# enhanced_data_dir = "/kaggle/working/enhanced_data"  # Enhanced dataset

# # Define Image Transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # Resize to match GVT input
#     transforms.RandomHorizontalFlip(),  
#     transforms.RandomRotation(15),  
#     transforms.ToTensor(),  
#     transforms.Normalize([0.5], [0.5])  # Normalize
# ])

# # Load Original and Enhanced Datasets
# train_dataset = ImageFolder(root=train_data_dir, transform=transform)
# enhanced_dataset = ImageFolder(root=enhanced_data_dir, transform=transform)

# # Combine Both Datasets
# combined_dataset = torch.utils.data.ConcatDataset([train_dataset, enhanced_dataset])

# # Data Loaders
# train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(ImageFolder(root="/kaggle/input/hmmmmm/data/test", transform=transform), batch_size=32, shuffle=False)

# print(f"✅ Loaded {len(combined_dataset)} training images")


In [ ]:
import timm
import torch.nn as nn
import torch.optim as optim

# Load Pretrained Model (Alternative to GVT)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=1)  # Single output neuron
 # Swin Transformer
device = torch.device("cpu")  # Temporarily switch to CPU
model.to(device)



# Define Loss and Optimizer
criterion = torch.nn.BCEWithLogitsLoss()  # Use for binary classification
# Handles multi-class classification

optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

print("✅ Model loaded successfully!")


In [ ]:
# from tqdm import tqdm

# num_epochs = 3  # Adjust based on dataset size

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     correct, total = 0, 0

#     for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
#         images, labels = images.to(device), labels.to(device)
    
#         optimizer.zero_grad()
#         outputs = model(images)  # Output shape: [batch_size, 1]

#         labels = labels.view(-1, 1).float()  # Reshape labels to [batch_size, 1] and convert to float
#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()


        
#         running_loss += loss.item()
#         _, predicted = torch.max(outputs, 1)
#         correct += (predicted == labels).sum().item()
#         total += labels.size(0)
    
#     train_acc = 100 * correct / total
#     print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")


In [ ]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
# labels = labels.to(device).float()
# criterion = torch.nn.BCEWithLogitsLoss()


# def evaluate(model, val_loader, criterion):
#     model.eval()
#     running_loss = 0.0
#     correct, total = 0, 0

#     with torch.no_grad():
#         for images, labels in tqdm(val_loader, desc="Evaluating"):
#             images, labels = images.to(device), labels.to(device).float()  # BCEWithLogitsLoss requires float labels
            
#             outputs = model(images).squeeze()
#             loss = criterion(outputs, labels)
#             running_loss += loss.item()
            
#             predicted = torch.round(torch.sigmoid(outputs))  # Convert logits to binary labels
#             correct += (predicted == labels).sum().item()
#             total += labels.size(0)

#     val_loss = running_loss / len(val_loader)
#     val_acc = 100 * correct / total
#     return {"val_loss": val_loss, "val_acc": val_acc}

# # Call evaluation function
# result = evaluate(model, test_dl, criterion)
# print(result)


In [ ]:
# from tqdm import tqdm
# import torch

# model.eval()  # Set model to evaluation mode
# correct, total = 0, 0
# running_loss = 0.0

# with torch.no_grad():  # No gradient calculation for testing
#     for images, labels in tqdm(test_loader, desc="Testing"):
#         images, labels = images.to(device), labels.to(device) # Ensure labels are float for BCEWithLogitsLoss
        
#         outputs = model(images).squeeze()  # Ensure output shape matches labels
#         loss = criterion(outputs, labels)
#         running_loss += loss.item()
        
#         # Binary classification fix: Use sigmoid + thresholding
#         predicted = torch.round(torch.sigmoid(outputs))
#         correct += (predicted == labels).sum().item()
#         total += labels.size(0)

# # Calculate test accuracy
# test_acc = 100 * correct / total
# print(f"✅ Test Loss: {running_loss/len(test_loader):.4f}, Test Accuracy: {test_acc:.2f}%")


In [ ]:
# # Save the model
# model_save_path = "/kaggle/working/pcos_classifier.pth"
# torch.save(model.state_dict(), model_save_path)

# print(f"✅ Model saved at {model_save_path}")


In [ ]:
# import torch
# from torch.utils.data import Dataset
# from PIL import Image
# import os

# class PCOSDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.image_paths = []
#         self.labels = []

#         # Iterate over the categories: "notinfected" and "infected"
#         for label, category in enumerate(["notinfected", "infected"]):
#             category_path = os.path.join(root_dir, category)

#             # Ensure the category folder exists
#             if not os.path.isdir(category_path):
#                 print(f"⚠️ Warning: Missing category folder {category_path}")
#                 continue  

#             for img_name in os.listdir(category_path):
#                 img_path = os.path.join(category_path, img_name)

#                 # Validate image before adding it
#                 if self.is_valid_image(img_path):
#                     self.image_paths.append(img_path)
#                     self.labels.append(label)
#                 else:
#                     print(f"❌ Skipping invalid image: {img_path}")

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path = self.image_paths[idx]
#         label = self.labels[idx]

#         image = Image.open(img_path).convert("RGB")  # Ensure all images are RGB

#         if self.transform:
#             image = self.transform(image)

#         return image, label

#     @staticmethod
#     def is_valid_image(img_path):
#         """Check if an image file is valid."""
#         try:
#             with Image.open(img_path) as img:
#                 img.verify()  # Verify image integrity
#             return True
#         except (IOError, OSError):
#             return False


In [ ]:
# from torchvision import transforms
# from torch.utils.data import DataLoader

# # Define transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # Resize to match model input
#     transforms.ToTensor(),  # Convert to PyTorch tensor
#     transforms.Normalize([0.5], [0.5])  # Normalize
# ])

# # # Load training and testing datasets
# # train_data = PCOSDataset(root_dir="/kaggle/input/hmmmmm/data/train", transform=transform)
# test_data = PCOSDataset(root_dir="/kaggle/input/hmmmmm/data/test", transform=transform)

# # # Create data loaders
# # train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# print(f"✅ Training samples: {len(train_data)}")
# print(f"✅ Testing samples: {len(test_data)}")


In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset
from torchvision.datasets import ImageFolder
import timm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

class ImageFolderEX(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except:
            return None
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

class SwinPCOSClassifier:
    def __init__(self, train_dir, test_dir, enhanced_dir, batch_size=32):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        self.batch_size = batch_size
        # Swin Transformer specific transforms
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Setup directories
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.enhanced_dir = enhanced_dir
        
        # Initialize model and training components
        self.model = self._create_model()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(), 
            lr=1e-4,
            weight_decay=0.05  # Weight decay helps prevent overfitting
        )
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer,
            T_max=10,  # Number of epochs
            eta_min=1e-6
        )
        
    def _create_model(self):
        # Using Swin Transformer base model
        model = timm.create_model(
            'swin_base_patch4_window7_224',
            pretrained=True,
            num_classes=2
        )
        model = model.to(self.device)
        return model
    
    def prepare_data(self):
        # Load datasets
        train_dataset = ImageFolderEX(self.train_dir, transform=self.transform)
        enhanced_dataset = ImageFolderEX(self.enhanced_dir, transform=self.transform)
        test_dataset = ImageFolderEX(self.test_dir, transform=self.transform)
        
        # Combine training datasets
        combined_dataset = ConcatDataset([train_dataset, enhanced_dataset])
        
        # Create data loaders
        self.train_loader = DataLoader(
            combined_dataset, 
            batch_size=self.batch_size, 
            shuffle=True,
            collate_fn=self.collate_fn,
            num_workers=2,
            pin_memory=True
        )
        
        self.test_loader = DataLoader(
            test_dataset, 
            batch_size=self.batch_size, 
            shuffle=False,
            collate_fn=self.collate_fn,
            num_workers=2,
            pin_memory=True
        )
        
        print(f"Training samples: {len(combined_dataset)}")
        print(f"Test samples: {len(test_dataset)}")
        print(f"Classes: {train_dataset.classes}")
        
    @staticmethod
    def collate_fn(batch):
        batch = list(filter(lambda x: x is not None, batch))
        return torch.utils.data.dataloader.default_collate(batch)
    
    def train(self, epochs=10):
        self.model.train()
        history = {
            'train_loss': [], 
            'train_acc': [],
            'learning_rates': []
        }
        
        best_acc = 0.0
        
        for epoch in range(epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            
            pbar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{epochs}')
            for batch_idx, (inputs, labels) in enumerate(pbar):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                # Forward pass
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                # Backward pass
                loss.backward()
                self.optimizer.step()
                
                # Statistics
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                # Update progress bar
                pbar.set_postfix({
                    'loss': running_loss/(batch_idx+1),
                    'acc': 100.*correct/total,
                    'lr': self.optimizer.param_groups[0]['lr']
                })
            
            # Update learning rate
            self.scheduler.step()
            
            # Calculate epoch statistics
            epoch_loss = running_loss/len(self.train_loader)
            epoch_acc = 100.*correct/total
            
            # Save history
            history['train_loss'].append(epoch_loss)
            history['train_acc'].append(epoch_acc)
            history['learning_rates'].append(self.optimizer.param_groups[0]['lr'])
            
            # Save best model
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                self.save_model('best_swin_pcos_model.pth')
            
            print(f'\nEpoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%')
        
        return history
    
    def evaluate(self):
        self.model.eval()
        all_preds = []
        all_labels = []
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(self.test_loader, desc='Evaluating'):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # Calculate metrics
        test_accuracy = 100.*correct/total
        test_loss = test_loss/len(self.test_loader)
        
        print(f'\nTest Loss: {test_loss:.4f}')
        print(f'Test Accuracy: {test_accuracy:.2f}%')
        
        # Detailed metrics
        report = classification_report(
            all_labels, 
            all_preds, 
            target_names=['Not Infected', 'Infected'],
            digits=4
        )
        conf_matrix = confusion_matrix(all_labels, all_preds)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            conf_matrix, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=['Not Infected', 'Infected'],
            yticklabels=['Not Infected', 'Infected']
        )
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
        print("\nClassification Report:")
        print(report)
        
        return test_accuracy, test_loss
    
    def save_model(self, path):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict()
        }, path)
        print(f"Model saved to {path}")
    
    def load_model(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        print(f"Model loaded from {path}")

In [ ]:
# # Initialize the classifier
# classifier = SwinPCOSClassifier(
#     train_dir="/kaggle/input/hmmmmm/data/train",
#     test_dir="/kaggle/input/hmmmmm/data/test",
#     enhanced_dir="/kaggle/working/enhanced_data",
#     batch_size=12  # Smaller batch size due to larger model
# )

# # Prepare the datasets
# classifier.prepare_data()

# # Train the model
# history = classifier.train(epochs=8)

# # Evaluate the model
# test_accuracy, test_loss = classifier.evaluate()

In [1]:

#train and test data directory
data_dir = "/kaggle/input/hmmmmm/data/train"
test_data_dir = "/kaggle/input/hmmmmm/data/test"



transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor()])
 
test_ds = ImageFolderEX(test_data_dir, transform = transform)
 
print('test samples:', len(test_ds))
import os
import cv2
import numpy as np
import random
import shutil
from tqdm import tqdm
from skimage import exposure
from skimage.filters import unsharp_mask

class ImageFolderEX(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except:
            return None
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

# Original dataset path
data_dir = "/kaggle/input/hmmmmm/data/train"
enhanced_directory = "/kaggle/working/enhanced_data"  # Save enhanced images

# Create directory for enhanced images
if not os.path.exists(enhanced_directory):
    os.makedirs(enhanced_directory)

# Define augmentation functions
def image_translation(img, tx, ty):
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

def image_reflection(img):
    return cv2.flip(img, 1)  # Flip horizontally

def image_rotation(img, angle):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
    return cv2.warpAffine(img, M, (w, h))

def image_scaling(img, scale):
    h, w = img.shape[:2]
    return cv2.resize(img, (int(w * scale), int(h * scale)))

def image_cropping(img, x1, y1, x2, y2):
    return img[y1:y2, x1:x2]

def image_shear_x(img, shear_factor):
    h, w = img.shape[:2]
    M = np.array([[1, shear_factor, 0], [0, 1, 0]], dtype=np.float32)
    return cv2.warpAffine(img, M, (w, h))

def image_shear_y(img, shear_factor):
    h, w = img.shape[:2]
    M = np.array([[1, 0, 0], [shear_factor, 1, 0]], dtype=np.float32)
    return cv2.warpAffine(img, M, (w, h))

# Multi-Scale Retinex Enhancement
def multi_scale_retinex(img, sigma_list=[15, 80, 250], gain=128, offset=128):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.float32(img) + 1.0

    retinex = np.zeros_like(img)
    for sigma in sigma_list:
        retinex += np.log1p(img) - np.log1p(cv2.GaussianBlur(img, (0, 0), sigma))

    retinex = (retinex - np.min(retinex)) / (np.max(retinex) - np.min(retinex)) * gain + offset
    retinex = np.clip(retinex, 0, 255).astype(np.uint8)
    
    return cv2.cvtColor(retinex, cv2.COLOR_GRAY2BGR)

# Process images
for category in ["infected", "notinfected"]:
    category_path = os.path.join(data_dir, category)
    enhanced_category_path = os.path.join(enhanced_directory, category)

    # Create directories
    if not os.path.exists(enhanced_category_path):
        os.makedirs(enhanced_category_path)

    for img_name in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue  # Skip unreadable images

        # Apply augmentations
        transformed_images = [
            image_translation(img, random.randint(-50, 50), random.randint(-30, 30)),
            image_reflection(img),
            image_rotation(img, random.randint(-45, 45)),
            image_scaling(img, random.uniform(0.8, 1.2)),
            image_cropping(img, 50, 50, img.shape[1]-50, img.shape[0]-50),
            image_shear_x(img, random.uniform(-0.3, 0.3)),
            image_shear_y(img, random.uniform(-0.3, 0.3)),
        ]

        # Save augmented images
        for i, aug_img in enumerate(transformed_images):
            aug_img_path = os.path.join(enhanced_category_path, f"{img_name}_aug_{i}.jpg")
            cv2.imwrite(aug_img_path, aug_img)

        # Apply MSR enhancement and save
        enhanced_img = multi_scale_retinex(img)
        enhanced_img_path = os.path.join(enhanced_category_path, f"{img_name}_enhanced.jpg")
        cv2.imwrite(enhanced_img_path, enhanced_img)

print("✅ Dataset augmentation and enhancement complete! 🚀")


NameError: name 'transforms' is not defined

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset
from torchvision.datasets import ImageFolder
import timm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import numpy as np
import random

class ImageFolderEX(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except:
            return None
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

class SwinPCOSClassifier:
    def __init__(self, train_dir, test_dir, enhanced_dir, batch_size=32):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.enhanced_dir = enhanced_dir
        
        self.model = self._create_model()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(), 
            lr=1e-4,
            weight_decay=0.05
        )
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer,
            T_max=10,
            eta_min=1e-6
        )
        
    def _create_model(self):
        model = timm.create_model(
            'swin_base_patch4_window7_224',
            pretrained=True,
            num_classes=2
        )
        model = model.to(self.device)
        return model
    
    def prepare_data(self):
        train_dataset = ImageFolderEX(self.train_dir, transform=self.transform)
        enhanced_dataset = ImageFolderEX(self.enhanced_dir, transform=self.transform)
        test_dataset = ImageFolderEX(self.test_dir, transform=self.transform)
        
        combined_dataset = ConcatDataset([train_dataset, enhanced_dataset])
        
        self.train_loader = DataLoader(
            combined_dataset, 
            batch_size=self.batch_size, 
            shuffle=True,
            collate_fn=self.collate_fn,
            num_workers=2,
            pin_memory=True
        )
        
        self.test_loader = DataLoader(
            test_dataset, 
            batch_size=self.batch_size, 
            shuffle=False,
            collate_fn=self.collate_fn,
            num_workers=2,
            pin_memory=True
        )
        
        print(f"Training samples: {len(combined_dataset)}")
        print(f"Test samples: {len(test_dataset)}")
        print(f"Classes: {train_dataset.classes}")
        
    @staticmethod
    def collate_fn(batch):
        batch = list(filter(lambda x: x is not None, batch))
        return torch.utils.data.dataloader.default_collate(batch)
    
    def train(self, epochs=10):
        self.model.train()
        history = {
            'train_loss': [], 
            'train_acc': [],
            'learning_rates': []
        }
        
        best_acc = 0.0
        
        for epoch in range(epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            
            pbar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{epochs}')
            for batch_idx, (inputs, labels) in enumerate(pbar):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                loss.backward()
                self.optimizer.step()
                
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                pbar.set_postfix({
                    'loss': running_loss/(batch_idx+1),
                    'acc': 100.*correct/total,
                    'lr': self.optimizer.param_groups[0]['lr']
                })
            
            self.scheduler.step()
            
            epoch_loss = running_loss/len(self.train_loader)
            epoch_acc = 100.*correct/total
            
            history['train_loss'].append(epoch_loss)
            history['train_acc'].append(epoch_acc)
            history['learning_rates'].append(self.optimizer.param_groups[0]['lr'])
            
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                self.save_model('best_swin_pcos_model.pth')
            
            print(f'\nEpoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%')
        
        return history
    
    def evaluate(self):
        self.model.eval()
        all_preds = []
        all_labels = []
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(self.test_loader, desc='Evaluating'):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        test_accuracy = 100.*correct/total
        test_loss = test_loss/len(self.test_loader)
        
        print(f'\nTest Loss: {test_loss:.4f}')
        print(f'Test Accuracy: {test_accuracy:.2f}%')
        
        report = classification_report(
            all_labels, 
            all_preds, 
            target_names=['Not Infected', 'Infected'],
            digits=4
        )
        conf_matrix = confusion_matrix(all_labels, all_preds)
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            conf_matrix, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=['Not Infected', 'Infected'],
            yticklabels=['Not Infected', 'Infected']
        )
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
        print("\nClassification Report:")
        print(report)
        
        return test_accuracy, test_loss
    
    def save_model(self, path):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict()
        }, path)
        print(f"Model saved to {path}")
    
    def load_model(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        print(f"Model loaded from {path}")

if __name__ == "__main__":
    classifier = SwinPCOSClassifier(
        train_dir="/kaggle/input/hmmmmm/data/train",
        test_dir="/kaggle/input/hmmmmm/data/test",
        enhanced_dir="/kaggle/working/enhanced_data",
        batch_size=12
    )
    
    classifier.prepare_data()
    history = classifier.train(epochs=8)
    test_accuracy, test_loss = classifier.evaluate()
